In [1]:
using Pkg
using CSV
using DataFrames
using Printf
using Base.Threads
Pkg.add("CSV")
Pkg.add("DataFrames")

    Updating registry at `C:\Users\Gustavo\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Gustavo\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Gustavo\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Gustavo\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Gustavo\.julia\environments\v1.11\Manifest.toml`


In [2]:
# Cargar el archivo ratings en CSV en un DataFrame
df_ratings = CSV.read("Documents/ml-25m/ratings.csv", DataFrame)

Row,userId,movieId,rating,timestamp
,Int64,Int64,Float64,Int64
1,1,296,5.0,1147880044
2,1,306,3.5,1147868817
3,1,307,5.0,1147868828
4,1,665,5.0,1147878820
5,1,899,3.5,1147868510
6,1,1088,4.0,1147868495
7,1,1175,3.5,1147868826
8,1,1217,3.5,1147878326
9,1,1237,5.0,1147868839


In [3]:
function dividir_dataframe(df::DataFrame, num_partes::Int)
    particiones = DataFrame[]
    nrows = nrow(df)
    filas_por_parte = div(nrows, num_partes)

    for i in 1:num_partes
        start_row = (i-1) * filas_por_parte + 1
        end_row = i * filas_por_parte
        if i == num_partes
            end_row = nrows
        end
        push!(particiones, df[start_row:end_row, :])
    end
    
    return particiones
end

# Dividir el DataFrame
num_procesadores = Sys.CPU_THREADS
partes_df = dividir_dataframe(df_ratings, num_procesadores)

# Imprimir información sobre las partes
for i in 1:length(partes_df)
    @printf("Parte %d tiene %d renglones\n", i, nrow(partes_df[i]))
end

Parte 1 tiene 2083341 renglones
Parte 2 tiene 2083341 renglones
Parte 3 tiene 2083341 renglones
Parte 4 tiene 2083341 renglones
Parte 5 tiene 2083341 renglones
Parte 6 tiene 2083341 renglones
Parte 7 tiene 2083341 renglones
Parte 8 tiene 2083341 renglones
Parte 9 tiene 2083341 renglones
Parte 10 tiene 2083341 renglones
Parte 11 tiene 2083341 renglones
Parte 12 tiene 2083344 renglones


In [4]:
function guardar_partes_csv(partes_df::Vector{DataFrame}, base_filename::String)
    @threads for i in 1:length(partes_df)
        filename = @sprintf("%s_%02d.csv", base_filename, i)
        CSV.write(filename, partes_df[i])
        println("Archivo generado: ", filename, " con ", nrow(partes_df[i]), " filas.")
    end
end

# Guardar las partes en archivos CSV separados
guardar_partes_csv(partes_df, "ratings")


Archivo generado: ratings_01.csv con 2083341 filas.
Archivo generado: ratings_02.csv con 2083341 filas.
Archivo generado: ratings_03.csv con 2083341 filas.
Archivo generado: ratings_04.csv con 2083341 filas.
Archivo generado: ratings_05.csv con 2083341 filas.
Archivo generado: ratings_06.csv con 2083341 filas.
Archivo generado: ratings_07.csv con 2083341 filas.
Archivo generado: ratings_08.csv con 2083341 filas.
Archivo generado: ratings_09.csv con 2083341 filas.
Archivo generado: ratings_10.csv con 2083341 filas.
Archivo generado: ratings_11.csv con 2083341 filas.
Archivo generado: ratings_12.csv con 2083344 filas.


In [5]:
function FindRatingsMaster()
    nF = 12 # number of files with ratings
    kg = ["Action", "Adventure", "Animation", "Children", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "IMAX", "Musical", "Mystery", "Romance","Sci-Fi", "Thriller", "War", "Western", "(no genres listed)"]
    ng = size(kg, 1)
    ra = zeros(ng, nF)
    ca = zeros(ng, nF)
    
    dfm = CSV.read("Documents/ml-25m/movies.csv", DataFrame)
    dfm = dfm[:, [:movieId, :genres]]
    
    dfr_v = [DataFrame() for _ in 1:nF]
    
    Threads.@threads for i = 1:nF
        rfn = "ratings_" * @sprintf("%02d", i) * ".csv"
        println(rfn)
        dfr_v[i] = CSV.read(rfn, DataFrame)
        ra[:,i], ca[:,i] = FindRatingsWorker(i, ng, kg, dfm, dfr_v[i])
    end
    
    sra = zeros(ng)
    sca = zeros(ng)
    
    for i = 1:ng
        for j = 1:nF
            sra[i] += ra[i, j]
            sca[i] += ca[i, j]
        end
    end

    # Calcular el promedio de ratings por género
    promedio_ra = sra ./ sca

    @sync for i = 1:ng
        @printf("ca = %d   ra = %.5f   promedio = %.5f   genre = %s  \n", sca[i], sra[i], promedio_ra[i], kg[i])
    end
    return sca, promedio_ra, kg
end 

function FindRatingsWorker(w::Integer, ng::Integer, kg::Array, dfm::DataFrame, dfr::DataFrame)
    println("In Worker ", w, "\n")

    ra = zeros(ng) # ra is an 1D array for keeping the sum of the ratings for each genre
    ca = zeros(ng) # ca is an 1D array to keep the number of ratings for each genre

    println("local ndfr after resize =", size(dfr, 1))

    # The inner join will have the following columns: {movieId, genre, rating}
    ij = innerjoin(dfm, dfr, on=:movieId)
    nij = size(ij, 1)

    for i = 1:ng
        for j = 1:nij
            r = ij[j,:] # get all columns for row j, genre is col=2 of the row
            g = r[:genres]
            if contains(g, kg[i])
                ca[i] += 1    # keep the count of ratings for this genre
                ra[i] += r[:rating] # add the value for this genre
            end
        end
    end

    return ra, ca
end

start_time = time()
sca, promedio_ra, kg = FindRatingsMaster()
end_time = time()
println("Tiempo total de ejecución: ", end_time - start_time, " segundos")


ratings_01.csv
In Worker 1

local ndfr after resize =2083341
ratings_02.csv
In Worker 2

local ndfr after resize =2083341
ratings_03.csv
In Worker 3

local ndfr after resize =2083341
ratings_04.csv
In Worker 4

local ndfr after resize =2083341
ratings_05.csv
In Worker 5

local ndfr after resize =2083341
ratings_06.csv
In Worker 6

local ndfr after resize =2083341
ratings_07.csv
In Worker 7

local ndfr after resize =2083341
ratings_08.csv
In Worker 8

local ndfr after resize =2083341
ratings_09.csv
In Worker 9

local ndfr after resize =2083341
ratings_10.csv
In Worker 10

local ndfr after resize =2083341
ratings_11.csv
In Worker 11

local ndfr after resize =2083341
ratings_12.csv
In Worker 12

local ndfr after resize =2083344
ca = 7446918   ra = 25815424.50000   promedio = 3.46659   genre = Action  
ca = 5832424   ra = 20515228.50000   promedio = 3.51744   genre = Adventure  
ca = 1630987   ra = 5895930.50000   promedio = 3.61495   genre = Animation  
ca = 2124258   ra = 7291531.50000  